## Long Short-Term Memory implementation

In [ ]:
from theano import *
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams
import pandas as pd
import numpy as np

### Loading data

In [ ]:
tsx = pd.read_csv('SPY.csv')

#difference and normalize feature
X = tsx.values[:,1:-2]
Xd = np.diff(X,axis=0)
Xsd = ((Xd - np.mean(Xd,axis=0)) / np.var(Xd,axis=0)).astype(np.float32)[:-1][-5365:]

#generate up/down label
Y = ((Xd[:,4] > 0)*1)[1:].astype(np.int32)[-5365:].reshape(-1,1)

### Building LSTM in theano

In [ ]:
#parameters
n_in = Xsd.shape[1]
n_blocks = 36
seed = 132345
activation = T.nnet.relu

######
#computational flow
rng = np.random.RandomState(seed)
x = T.matrix('x')
y = T.imatrix('y')
def genW(size):
    return np.asarray(
        rng.uniform(
            low=-0.1,
            high=0.1,
            size = size
        ),
        dtype = theano.config.floatX)
M = n_in
N = n_blocks
 #input weights
Wz = theano.shared(value=genW((M,N)),name="Wz",borrow=True)
Wi = theano.shared(value=genW((M,N)),name="Wi",borrow=True)
Wf = theano.shared(value=genW((M,N)),name="Wf",borrow=True)
Wo = theano.shared(value=genW((M,N)),name="Wo",borrow=True)
#recurrence weights
Rz = theano.shared(value=genW((N,N)),name="Rz",borrow=True)
Ri = theano.shared(value=genW((N,N)),name="Ri",borrow=True)
Rf = theano.shared(value=genW((N,N)),name="Rf",borrow=True)
Ro = theano.shared(value=genW((N,N)),name="Ro",borrow=True)
#peephole weights
pi = theano.shared(value=np.zeros(N,dtype=theano.config.floatX),name="pi",borrow=True)
pf = theano.shared(value=np.zeros(N,dtype=theano.config.floatX),name="pf",borrow=True)
po = theano.shared(value=np.zeros(N,dtype=theano.config.floatX),name="po",borrow=True)
#bias weights
bz = theano.shared(value=np.zeros(N,dtype=theano.config.floatX),name="bz",borrow=True)
bi = theano.shared(value=np.zeros(N,dtype=theano.config.floatX),name="bi",borrow=True)
bf = theano.shared(value=np.zeros(N,dtype=theano.config.floatX),name="bf",borrow=True)
bo = theano.shared(value=np.zeros(N,dtype=theano.config.floatX),name="bo",borrow=True)
params = [Wz,Wi,Wf,Wo,Rz,Ri,Rf,Ro,
          pi,pf,po,bz,bi,bf,bo]
#hidden states
y0 = theano.shared(value=np.zeros((1,N),dtype=theano.config.floatX),name="y0",borrow=True)
c0 = theano.shared(value=np.zeros((1,N),dtype=theano.config.floatX),name="c0",borrow=True)
#regression weights
W_out = theano.shared(value=genW((N,1)),name="W_out",borrow=True)
b_out = theano.shared(value=0.0,name="b_out",borrow=True)
params += [W_out, b_out]
#recurrence step
def recurrence(x_t,y_t1,c_t1):
    #block input
    z_t = activation(T.dot(x_t,Wz) + T.dot(y_t1,Rz) + bz)
    #input gate
    i_t = T.nnet.sigmoid(T.dot(x_t,Wi) + T.dot(y_t1,Ri) + pi*c_t1 + bi)
    #forget gate
    f_t = T.nnet.sigmoid(T.dot(x_t,Wf) + T.dot(y_t1,Rf) + pf*c_t1 + bf)
    #cell
    c_t = z_t*i_t + c_t1*f_t
    #output gate
    o_t = T.nnet.sigmoid(T.dot(x_t,Wo) + T.dot(y_t1,Ro) + po*c_t1 + bo)
    #block output
    y_t = activation(c_t) * o_t
    return [y_t, c_t], OrderedDict({y0: y_t, c0: c_t})
#scan for recurrence
[u_t, c_t], _ = theano.scan(fn=recurrence,
                             sequences=[x],
                             outputs_info=[y0,c0]
                            )
#lstm output state
lstm_out = u_t[:,0,:]
#output probability
p_y_x = T.nnet.sigmoid(T.dot(lstm_out,W_out) + b_out)
#label prediction
y_p = T.gt(p_y_x,0.5)*1
#cross entropy loss
CE = T.nnet.binary_crossentropy(p_y_x,y).mean()

### Training

In [ ]:
#parameters
tlength = 5000
learning_rate=0.01
n_epochs=2000
sX = theano.shared(np.asarray(Xsd[:tlength], dtype=theano.config.floatX))
sY = theano.shared(np.asarray(Y[:tlength], dtype=np.int32))


####
#training function
cost = CE
gparams = [T.grad(cost,param) for param in params]
updates = [(param, param - learning_rate * gparam) for param,gparam in zip(params,gparams)]
train_lstm = theano.function(
    inputs=[],
    outputs=[cost],
    updates=updates,
    givens={
        x: sX,
        y: sY
    },
)
####
#predicting function
pred_y = theano.function(
    inputs=[x],
    outputs=y_p
)

In [ ]:
#training
print "training..."
for ep in range(n_epochs):
        print('Training epoch %d, cost %f' % (ep, np.mean(train_lstm())))

In [ ]:
#testing
print np.mean(pred_y(Xsd)[tlength:]==Y[tlength:])